In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:

from __future__ import print_function
import numpy
import matplotlib.pyplot as plt
import warnings
import csv
import math
import os
import scipy
from scipy.signal import butter, lfilter, freqz
import math
from skimage.measure import label
from skimage import data, util
from skimage.measure import regionprops
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from numpy.linalg import solve
import xlrd


def angle(x,y):
    #return the angle between input vector and x axis
    angle1=math.atan2(y,x)
    return angle1/math.pi*180
    
def angle360(x1,y1,x2,y2):
    #return the angle between input vector1 and vector 2, 
    a1=math.atan2(y1,x1)
    a2=math.atan2(y2,x2)
    a=a1-a2
    a=round(a/math.pi*180)
    if a<0:
        a=a+360
    return a 

def angle180(x1,y1,x2,y2):
    #return the angle between input vector1 and vector 2, 
    #angel has maximum of 180 degrees as direction is neglected 
    a1=math.atan2(y1,x1)
    a2=math.atan2(y2,x2)
    a=a1-a2
    a=round(a/math.pi*180)
    if a<0:
        a=-a
    if a>180:
        a=a-180  
    return a


'''def csvreader(filename):
    #open csv file and return the number data in a numpy matrix
    with open(filename)as f:
        reader=csv.reader(f)
        header_row=next(reader)
        data=[]
        for line in reader:
            data.append(line)
    data=numpy.array(data)
    numdata=numpy.zeros(data.shape)
    for i in range(3,data.shape[0]):
        for j in range(data.shape[1]):
            numdata[i,j]=float(data[i,j])
            
    return numdata
   ''' 
  filename
  df= pd.read_csv(filename, skip_rows=3)
def csvreader(filename):
     '''with open(filename) as f: 
       reader=csv.reader(f)
       header_row=next(reader)
       for row in reader[3:]:
         data.append(row)
       data=numpy.array(data)
       numdata=numpy.zeros(data.shape)
       for i in range(0, data.shape[0]):
         for j in range(data.shape[1]):
           numdata[i,j]=float(data[i,j])
           '''
  filename
  df= pd.read_csv(filename, skip_rows=3)
  wit
  for row in reader:
    data.append(row)
    data=numpy.array(data)
    numdata=numpy.zeros(data.shape)
    for i in range (0, data.shape[0]):
      for j in range(data.shape[1]):
        numdata[i,j]=float(data[i,j])

         
       


def D1_2(f):
    """ calculate consistent 2nd order first derivative of a function func(x) N evenly spaced points"""

    N=len(f)
    dt = 1/30
    f_prime = numpy.zeros(N)
    f_prime[0] = f[:3].dot(numpy.array([-3, 4, -1]))/(2*dt)
    f_prime[1:-1] = (f[2:N] - f[0:-2])/(2*dt)
    f_prime[-1] = f[-3:].dot(numpy.array([1, -4, 3]))/(2*dt)
    
    return f_prime

def D2(f):
    """ calculate consistent 2nd order second derivative of a function func(x) defined on the interval [x_min,xmax]
    and sampled at N evenly spaced points"""
    dt=1/30
    D2f = numpy.zeros(x.shape) 
    D2f[1:-1] = (f[:-2] - 2*f[1:-1] + f[2:])/(dt**2)
    # patch up end points to be 1 sided 2nd derivatives
    D2f[0] = D2f[1]
    D2f[-1] = D2f[-2]

    
    return D2f


def plt_trajactory(numdata,center_x=730,center_y=575):
    '''
    plot the trajectory of mice midpoint for all the given data
    
    '''
    fig=plt.figure(figsize=(4,4))
    axes=fig.add_subplot(1,1,1)

    axes.plot(numdata[:,16],numdata[:,17],'bo',label='midpoint track')
    
    axes.grid()
    #axes.plot(numdata[i,16],numdata[i,17],'ro',label='selected point')
    if center_x == 'average':
        center_x=(min(numdata[:,16])+max(numdata[:,16]))/2
        center_y=(min(numdata[:,17])+max(numdata[:,17]))/2
    centertext='x= '+str(center_x)+', y= '+str(center_y)
    axes.text(800,800,centertext)
    axes.plot(center_x,center_y,'rx',label='center')
    axes.legend(loc='best')
    axes.set_xlim([0,1200])
    axes.set_ylim([0,1200])
    return axes

def lp_filter(signal):
    #construct a low frequency pass filter to smooth a signal(1;N vector)
    order = 5

    sampling_freq = 30

    cutoff_freq = 2

    sampling_duration = 5

    number_of_samples = len(signal)

    normalized_cutoff_freq = 2 * cutoff_freq / sampling_freq

    numerator_coeffs, denominator_coeffs = scipy.signal.butter(order, normalized_cutoff_freq)

    filtered_signal = scipy.signal.lfilter(numerator_coeffs, denominator_coeffs, signal)

    return filtered_signal

def lp_filterm(numdata):
    #construct a low frequency pass filter to smooth colums signals in a matrix(M;N matrix)
    fnumdata=numpy.zeros(numdata.shape)
    for j in range(1,numdata.shape[1]):
        signal=numdata[:,j]
        fnumdata[:,j]=lp_filter(signal)
    return fnumdata


def findallfile(base):
     #return the file path list of all files in csv formats
    #base: search path root
    for root,ds,fs in os.walk(base):
        for f in fs:
            if f.endswith('.csv'):
                fullname=os.path.join(root,f)
                yield fullname
def findallfile_csv(base):
     #return the file path list of all files in csv formats
    #base: search path root
    pathlist=[]
    for root,ds,fs in os.walk(base):
        for f in fs:
            if f.endswith('.csv'):
                fullname=os.path.join(root,f)
                pathlist.append(fullname)
    return pathlist

def histgramplot(array,segment=1,maxh=100):
    # for the plot of distribution histgrame
    # array: input samples,
    # segment: the segment width of samples( the width of each bar in the histograme plot
    histnum=round(maxh/segment)
    hist=numpy.zeros(histnum)
    for i in array:
        if i>maxh:
            continue
        else:
            
            hist[int(i/segment)]=hist[int(i/segment)]+1
    x=numpy.linspace(0,maxh,histnum)
    hist=hist/len(array)
    return hist,x

def histgramplot2(array,segment=1,maxh=100):
    #same to histgramplot, only the output is not normalized by the total sample num
    histnum=round(maxh/segment)
    hist=numpy.zeros(histnum)
    for i in array:
        if i>maxh:
            continue
        else:
            
            hist[int(i/segment)]=hist[int(i/segment)]+1
    #x=numpy.linspace(0,maxh,histnum)
    x=numpy.arange(0,maxh,maxh/histnum)+0.5*maxh/histnum
    return hist,x

def survivorcurve(array,xmax,dx=1):
    # for the plot of survivor cuirve
    x=numpy.arange(0,xmax,dx)
    y=numpy.zeros(len(x))
    for j,xi in enumerate(x):
        y[j]=len([i for i in array if i>xi])/len(array)
                  
    return x,y




def findallfile_h5(base):
    #return the file path list of all files in H5 formats
    #base: search path root
    pathlist=[]
    for root,ds,fs in os.walk(base):
        for f in fs:
            if f.endswith('.h5'):
                fullname=os.path.join(root,f)
                pathlist.append(fullname)
    return pathlist

def re_eye(i):
    # create a Inverse diagonal matrix, used in construction of structure element when trying to find the center of the maze
    x = numpy.eye(i)
    #print(x)
    for line in x:
        length = len(x)
        for i in range(int(length / 2)):
            line[i], line[length - 1 - i] = line[length - 1 - i], line[i]
  
    #print(x)
    x=numpy.matrix(x,numpy.uint8)
    return x

def startframe(data):
    # return the frame number when the mouse is put in the maze
    #it based on the fact that likelyhood of tracking result is low and unstable before the mouse start exploring 
    #input is the raw number matrix with x,y and likelyhood
    likehooddata=data[:,(3,6,12,15)] # just use the likelihoods of ears, tailbase and nose, we should probs increase
    startf=0
    stablef=0
    pthresh=0.95 #also can go higher 
    while(stablef<100):
        if likehooddata[startf,0]>pthresh and likehooddata[startf,1]>pthresh and likehooddata[startf,2]>pthresh:

            stablef=stablef+1
        else:
            stablef=0
        startf=startf+1
        #print(startf)
    startf=startf-stablef+10
    return startf

def startframe_l(likehooddata):
    # return the frame number when the mouse is put in the maze
    #it based on the fact that likelyhood of tracking result is low and unstable before the mouse start exploring 
    #input only contain likelyhood array
    startf=0
    stablef=0
    pthresh=0.6 #could be made higher 
    while(stablef<100):
        if likehooddata[startf,0]>pthresh and likehooddata[startf,1]>pthresh and likehooddata[startf,2]>pthresh:

            stablef=stablef+1
        else:
            stablef=0
        startf=startf+1
        #print(startf)
    startf=startf-stablef+10
    return startf

def mazecenter(mx,my,plot=False):
    '''
    return the maze center of the experiment based on the trajectory of mouse,
    use dilation and erosion to connect the trajectory in to morphology shape and recognize the cross point of two main arms.
    
   !!!only work when the maze is presented as 'X' shape in videos, each arm should be at approximately 45,135,225,315 degrees,
   !!!If in the future want to analyze on + shape, you may alter the structure element of erosion from diagonal matrix to horizontal and vertical line. Or maybe use other ideas
    
    input: mx, the x coordinates array of mouse body center
        my, the y coordinates array of mouse body center
        plot, whether to plot the maze center finding result, enable researchers to double check on the result
    output: maze_center_x, maze_center_y
    
    '''
    width=int(max(mx))+1
    high=int(max(my))+1
    trajecI=numpy.zeros((width,high))

    for i in range(len(mx)):
        trajecI[int(mx[i]),int(my[i])]=1
    kernel = numpy.ones((10,10),numpy.uint8)
    dilation = cv2.dilate(trajecI,kernel,iterations = 1)
    kernel = numpy.ones((10,10),numpy.uint8)
    erode= cv2.erode(dilation,kernel,iterations = 1)
    kernel = numpy.ones((10,10),numpy.uint8)
    dilation2 = cv2.dilate(erode,kernel,iterations = 1)
    kernel2=numpy.identity(10,numpy.uint8)
    erode2= cv2.erode(dilation2,kernel2,iterations = 4)
    kernel3=re_eye(10)
    erode3= cv2.erode(dilation2,kernel3,iterations = 4)
    props_x=imlableprops(dilation2)
    props_x[0].centroid
    props_l1=imlableprops(erode2)
    props_l1[0].centroid
    props_l2=imlableprops(erode3)
    props_l2[0].centroid
    k_1=math.tan(props_l1[0].orientation)
    k_2=math.tan(props_l2[0].orientation)
    x0_1=props_l1[0].centroid[0]
    y0_1=props_l1[0].centroid[1]
    x0_2=props_l2[0].centroid[0]
    y0_2=props_l2[0].centroid[1]

    a=numpy.mat([[k_1,-1],[k_2,-1]])#
    b=numpy.mat([k_1*x0_1-y0_1,k_2*x0_2-y0_2]).T  #
    cross_xy=solve(a,b)

    if plot:
        fig=plt.figure(figsize=(15,14 ))
        axes=fig.add_subplot(1,1,1)
        axes.plot(mx,my,'o')
        x=numpy.linspace(400,1000,1000)
        y=math.tan(props_l1[0].orientation)*(x-props_l1[0].centroid[0])+props_l1[0].centroid[1]
        axes.plot(x,y,'r-')

        x=numpy.linspace(400,1000,1000)
        y=math.tan(props_l2[0].orientation)*(x-props_l2[0].centroid[0])+props_l2[0].centroid[1]
        axes.plot(x,y,'r-',label='Estimate arm direction')
        axes.plot(props_l1[0].centroid[0],props_l1[0].centroid[1],'rx')
        axes.plot(props_l2[0].centroid[0],props_l2[0].centroid[1],'rx')
        axes.plot(props_x[0].centroid[0],props_x[0].centroid[1],'kx',markersize=30)
        axes.plot(cross_xy[0],cross_xy[1],'r+',markersize=30,label='Estimate maze center')
        plt.legend(loc='best')
    return cross_xy[0],cross_xy[1]

def imlableprops(image):
    # return the properties of connectied area in binary image
    # used in function <mazecenter> to help find the center of the maze
    label_img = label(image, connectivity = image.ndim)
    props = regionprops(label_img)
    
    if len(props)>1:
        while len(props)>1:
            for i in range(len(props)):
                #print(props)
                if props[i].area<5000:
                    del(props[i])
                    break
    return props

def SA_counter(mx,my,center_x,center_y,disthresh=150,dismargin=20,EntryTimeLimit=False):
    '''
    Used to count alternation sequence of mouse in Spontaneous alternation experiments of x shape maze
    !!!only work when the maze is placed as X shape in video, the arm should not be vertical nor horizontal, 
    as four quarter origined from maze center is used to determine which arm the mouse is entering

    input: 
    mx: the x coordinate array of mouse(usually body center)
    my: the x coordinate array of mouse(usually body center)

    center_x: the x coordinate of maze center, default setted in function
    center_y: the y coordinate of maze center, default setted in function

    disthresh: the distance threshold to determine whether the mouse has entered the arm
    dismargin: the distance margin between threshold line, in case the mouse hesitating on the threshold is counted as several entries
    EntryTimeLimit: if set to true, when the mouse stay time in a arm is shorted than EntryTimeLimit, this alternation will not be counted,
           if set to false, all entries that meet the distance requirements would be counted


    output:
    en_num: return the entry axis sequence listed from 1-4, cooresponding to four axies in video as counter clock wise order.
     eg: [1,3,2,4] means the mouse visited 1,3,2,4 four arms of maze. 
    en_tf;  return a array indicating whether the mouse is making right decision(not repeated with previous 3 alternation)
       eg:[1 ,0 ,1] means the mouse make one correct, make one wrong decision and make a correct one in the end.
    en_f1: return the frame number the mouse entered the maze
    en_f2: return the frame number the mouse leaves the maze, divided by frame rate leads to time in seconds.

    '''
        
    en_state=0
    en_num=[5]
    en_tf=[]
    en_f1=[] 
    en_f2=[]

    deletenum=0

    #count all the entries
    for i in range(len(mx)):
        dis=numpy.linalg.norm([mx[i]-center_x,my[i]-center_y])
        x_sign=mx[i]-center_x
        y_sign=my[i]-center_y
        if not en_state:
            if dis>(disthresh+dismargin):
                if x_sign>=0 and y_sign>=0 and en_num[-1]!=1:
                    en_num.append(1)
                    en_state=1
                    en_f1.append(i)
                if x_sign>=0 and y_sign<0 and en_num[-1]!=2:
                    en_num.append(2)
                    en_state=1
                    en_f1.append(i)
                if x_sign<0 and y_sign<=0 and en_num[-1]!=3:
                    en_num.append(3)
                    en_state=1
                    en_f1.append(i)
                if x_sign<0 and y_sign>0 and en_num[-1]!=4:
                    en_num.append(4)
                    en_state=1
                    en_f1.append(i)             

        if en_state:
            if dis<(disthresh-dismargin):
                en_state=0
                en_f2.append(i)  

        if EntryTimeLimit:
            if len(en_f2)>1 and len(en_f1)>1 and len(en_f2)==len(en_f1):
                if (en_f2[-1]-en_f1[-1])<EntryTimeLimit:
                    del en_num[-1] 
                    del en_f1[-1]
                    del en_f2[-1]
                    deletenum=deletenum+1

    del en_num[0]

    #determine whether the alternation is repeated with previous alternation, the first 3 altenation entries are also recorded in en_tf, 
    #but they are stored as zeros, please remember to remove first 3 from correct percentage analyze in the next step.
    en_tf=numpy.zeros(len(en_num))
    for i in range(3,len(en_num)):
        if en_num[i]!=en_num[i-1] and en_num[i]!=en_num[i-2] and en_num[i]!=en_num[i-3]:
            en_tf[i]=1
        else:
            en_tf[i]=0

    if EntryTimeLimit:
        if deletenum:
            print('SA_counter deleted ', deletenum,' times due to too short frame gap')

    return numpy.transpose(en_num),en_tf,en_f1,en_f2






IndentationError: ignored

In [ ]:
#please change this file path to the folder that containing the csv/h5 files(the track result of DLC)
ctrl_path = r'/content/drive/MyDrive/ADL Lab Work/18moELSproject/CSVs/Ctrl'
ELS_path =  '/content/drive/MyDrive/ADL Lab Work/18moELSproject/CSVs/ELS'

csvlist=findallfile_csv(ctrl_path)#the file list need to process

In [ ]:
#for index in numpy.arange(0,len(csvlist)):
for index in numpy.arange(0,5):# this needs to be edited
    file=csvlist[index]
    
    numdata=numpy.matrix(csvreader(file))#
    likehooddata=numdata[:,(4,7,10,13,16)]
    
    startf=startframe_l(likehooddata)#based on the validity of tracking to determine when the experiment start 
    if startf>1000:
        #check if the code may get wrong time point of start of experiment
        print('Warning: Mouse enter maze time point could be wrong, please check if mouse start explore at',startf/24,'seconds in',index,'video ',file[len(filepathbase):-1])
    
    
    mx=numpy.array(numdata[startf:,28]) #28 and 29 means the column number of body midpoint x and y coordinates
    my=numpy.array(numdata[startf:,29])

    #only include data since the start of experiments
    #print(file,len(mx)/60/30)
    center_x,center_y=mazecenter(mx,my,plot=True)#return the maze center by processing binary image of mouse trajectory
    
    en_num,en_tf,en_f1,en_f2=SA_counter(mx,my,center_x,center_y,disthresh=150,dismargin=10,EntryTimeLimit=10)#count the alternation
    #en_num is the alternation arm sequence, f1 is the cooresponding enter time(frame num), f2 is the frame the mouse leave, 
    #en_tf representing whether the alternation is repeated(1 means correct,0 means repeating with previous three alternations)
    
    print('finish processing',file[len(ctrl_path):-1],'\n')

TypeError: ignored